In [23]:
import cobra
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('../../code/')
import leakage
from pathlib import Path
import matplotlib
import seaborn as sns
import scipy.stats as st
from scipy.stats import mannwhitneyu

In [24]:
def inf_to_100000(model):
    for r in model.reactions:
        if np.isinf(r.lower_bound):
            r.lower_bound = -100000
        if np.isinf(r.upper_bound):
            r.upper_bound = 100000
    return model

def print_summary(model, solution):
    # Print uptake
    for r in model.exchanges:
        flux = solution[r.id]
        if  flux >1e-3:
            print(r.id, r.name,flux, r.bounds)
        elif flux < -1e-3:
            print(r.id, r.name,flux, r.bounds)
    #print("## Demands ##")
    #for r in model.demands:
    #    print(r.id, solution[r.id])

    print("Growth: ", solution['r_2111'])
    print("Prot pool exchange: ", solution['prot_pool_exchange'])

# Load model

In [25]:
model = cobra.io.read_sbml_model('../../models/yeast/ecYeastGEM_batch_8_3_4.xml')
model.solver = 'gurobi'
model = inf_to_100000(model)

In [26]:
# Data on glucose uptake https://ami-journals.onlinelibrary.wiley.com/doi/10.1111/1751-7915.13995
init_glucose_conc = 16 #g/L
glucose_mw = 180.156 #g/mol
conc_gluc_mM = (init_glucose_conc/glucose_mw)*1e3
km_glc = 0.28 #mmol l−1
vmax_glc = 10 #mmol gDW−1

r_glc = vmax_glc * conc_gluc_mM / (conc_gluc_mM + km_glc)
print(r_glc)

9.968571784677293


## Note
As also stated in https://ami-journals.onlinelibrary.wiley.com/doi/10.1111/1751-7915.13995 glucose uptake is not constrained by km (i.e. vmax = 10)


In [28]:
model.reactions.get_by_id('r_1714_REV').upper_bound = 10


In [29]:
solution = model.optimize()
print_summary(model, solution)

r_1634 acetate exchange 3.1279933187031577 (0.0, 100000)
r_1672 carbon dioxide exchange 14.497501304355927 (0.0, 100000)
r_1761 ethanol exchange 9.481473517387105 (0.0, 100000)
r_1793 formate exchange 0.015479223367907058 (0.0, 100000)
r_2033 pyruvate exchange 1.8210776215830649 (0.0, 100000)
r_2100 water exchange 13.349287065068731 (0.0, 100000)
r_1654_REV ammonium exchange (reversible) 2.4743569591727215 (0.0, 100000)
r_1714_REV D-glucose exchange (reversible) 10.0 (0.0, 10)
r_1832_REV H+ exchange (reversible) 4.105673859636615 (0.0, 100000)
r_1992_REV oxygen exchange (reversible) 5.234950808334606 (0.0, 100000)
r_2005_REV phosphate exchange (reversible) 0.09876510324015143 (0.0, 100000)
r_2020_REV potassium exchange (reversible) 0.0013110424633373764 (0.0, 100000)
r_2049_REV sodium exchange (reversible) 0.0014338398290494171 (0.0, 100000)
r_2060_REV sulphate exchange (reversible) 0.03306272119862749 (0.0, 100000)
Growth:  0.36116872268247285
Prot pool exchange:  0.103720024310123


# Get leakage


In [30]:
exometabolites_folder = Path("../../data/yeast/")
leakage_df = leakage.get_leakage(exometabolites_folder, "yeast", time = 4)
leakage_df.set_index("Metabolite", inplace=True)
leakage_df.drop_duplicates(inplace=True)

In [35]:
leakage_df

,Leakage (mM/gDW/h)
Metabolite,
G6P,0.007276
F6P,0.012385
FBP,0.617192
DHAP,0.036482
23PG,0.002733
PEP,0.000261
PYR,-4.136282
RU5P+X5P,0.000429
CIT,1.070670


In [32]:
# Read metabolite mapping
mapping_df = pd.read_csv('../../data/id_mapping.csv', index_col=0)
df = pd.merge(leakage_df, mapping_df, left_index=True, right_index=True)
# df.set_index('met id', inplace = True)

In [34]:
mapping_df

,Metabolite name,Ecoli metabolite,Yeast metabolite
Abbreviation,,,
G6P,Glucose-6-phosphate,g6p,s_0568
F6P,Fructose-6-phosphate,f6p,s_0557
FBP,"Fructose-1,6-bisphosphate",fdp,s_0555
DHAP,Dihydroxyacetonephosphate,dhap,s_0629
GA3P,Glyceraldehyde-3-phosphate,g3p,s_0764
...,...,...,...
NaN,Methionine,NaN,NaN
NaN,Lysine,NaN,NaN
NaN,Tryptophane,NaN,NaN


# Plot correlation

In [11]:
solution = model.optimize()

In [ ]:
shadow_prices = []
log_leakage = []
labels = []
for met_id, row in leakage_df.iterrows():
    # Hack for ru5p__D
    met_id = met_id.split(',')[0]
    if row['Leakage (uM/h/OD)'] > 0:
        shadow_prices.append(-1*solution.shadow_prices['{0}_c'.format(met_id)])
        log_leakage.append(np.log(row['Leakage (uM/h/OD)']))
        labels.append(met_id)

